<a href="https://colab.research.google.com/github/danielahernandez29/Curso-IA/blob/main/Multicapa/Popularidad_de_una_canci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Regresión con Red Neuronal para predecir Popularidad de Canciones
Usamos Keras para predecir un valor numérico (popularidad).
"""

import pandas as pd # Para manejar los datos.

# Cargar el conjunto de datos desde una URL
url = "https://raw.githubusercontent.com/mevangelista-alvarado/datasets/refs/heads/main/spotify_songs.csv"
# Cargo el dataset de Spotify.
df = pd.read_csv(url)

print(df.head())

# Seleccionar características
features = [
    'danceability', 'energy', 'key', 'loudness',
    'mode', 'speechiness', 'acousticness', 'instrumentalness',
    'liveness', 'valence', 'tempo', 'duration_ms',
]
X = df[features].values # Mis variables de entrada.


y = df['popularity'].values # Mi variable a predecir.

# Dividimos los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

# Divido 80% (train) y 20% (test).
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, random_state=42
)

# Escalamos las características para mejorar el rendimiento del modelo
from sklearn.preprocessing import StandardScaler

# Escalo los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definimos nuestra red neuronal multicapa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Ocupamos Adam con una tasa de aprendizaje personalizada
from tensorflow.keras.optimizers import Adam

# Tasa de aprendizaje estándar.
learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)

model.compile(
    optimizer=adam_optimizer,
    loss='mse',
    metrics=['mae'],
)

# Entrenamos el modelo
# Entreno por 50 épocas
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=50,
)

# Graficamos la función de pérdida
import matplotlib.pyplot as plt

# Grafico la pérdida
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.title('Función de pérdida durante el entrenamiento')
plt.show()

# Evaluamos el modelo en el conjunto de prueba
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"MAE en el conjunto test: {mae}") # Mi error promedio absoluto.

# Predecimos y comparamos con los valores reales
import pandas as pd

# Hago las predicciones y las comparo con los valores reales.
predictions = model.predict(X_test).flatten()
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(comparison.head())

# seleccionamos una canción para predecir su popularidad
nombre_cancion = "Beso"

# Busco un ejemplo específico para predecir.
canciones_df = df[df['track_name'].str.contains(nombre_cancion, case=False, na=False)]

print(f"Canciones encontradas:")
canciones_df[['track_name', 'artists', 'album_name']].head()

# indice a selecionar
i = 0
cancion = canciones_df.iloc[i]

X_input = cancion[features].values.reshape(1, -1)
X_input = scaler.transform(X_input)

# Predigo la popularidad de esa canción.
prediccion = model.predict(X_input)[0][0]
print(f"Canción: {cancion['track_name']} - {cancion['artists']}")
print(f"Popularidad real: {cancion['popularity']}")
print(f"Predicción: {prediccion:.2f}")

# Calculamos métricas adicionales para evaluar el rendimiento del modelo
from sklearn.metrics import r2_score, mean_squared_error

# Calculo el R² y el MSE para tener más métricas.
r2 = r2_score(y_test, predictions)
print(f'R²: {r2}')

mse = mean_squared_error(y_test, predictions)
print(f'MSE: {mse}')